<a href="https://colab.research.google.com/github/victrixgyasi/AVeriTeC_shared_task/blob/main/BART_BASE_finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install accelerate argparse nltk numpy rank_bm25 tqdm torch torchvision transformers peft subprocess32==3.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_c

In [ ]:
import numpy as np
import argparse
import json
import nltk
from rank_bm25 import BM25Okapi
import tqdm
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from accelerate import Accelerator
from transformers import BartForConditionalGeneration
from sklearn.model_selection import train_test_split
#from transformers import BartTokenizer
#from torch.utils.data import Dataset
import nltk
nltk.download('punkt')
nltk.download('wordnet')
# Mount on GDrive
from google.colab import drive
drive.mount('/content/drive')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Mounted at /content/drive


In [ ]:
### Load and preprocess the data
def open_data_file(file_name):
    """ Loads a JSON file
    """
    with open(file_name) as f:
        j = json.load(f)
        examples = j
        return examples

In [ ]:
### Define the dataset class
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        input_text = item['claim']
        target_texts = item['questions']

        input_encodings = self.tokenizer(
            input_text, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )

        target_encodings = self.tokenizer(
            target_texts, max_length=self.max_length, truncation=True, padding="max_length", return_tensors="pt"
        )

        input_ids = input_encodings.input_ids.squeeze()
        attention_mask = input_encodings.attention_mask.squeeze()
        target_ids = target_encodings.input_ids.squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": target_ids,
        }

### Initialise the model
bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

# Create datasets
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
train_dataset = CustomDataset(train_dataset, tokenizer)
test_dataset = CustomDataset(test_dataset, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

NameError: name 'train_dataset' is not defined

In [ ]:
model_load_path = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/results/trained_model'
model = BartForConditionalGeneration.from_pretrained(model_load_path)
tokenizer = BartTokenizer.from_pretrained(model_load_path)

In [ ]:
def get_claims(file_name):
    """
    Get just claims for dev set, so questions can be generated.
    Parameters:
    file_name (str): The name of the JSON file containing the data.
    Returns:
    claims (list): A list of dictionaries containing the claims and questions.
    """
    examples = open_data_file(file_name)
    claims = []
    for example in examples:
        claim = example["claim"]
        questions = [] # empty
        claim_dict_builder = {}
        claim_dict_builder["claim"] = claim
        claim_dict_builder["questions"] = questions
        claims.append(claim_dict_builder)
    return claims

def write_to_json_file(output_path:str, data, append = True):
    """
    Write data to a JSON file. If the file exists, it will be overwritten.

    Parameters:
    filename (str): The name of the file where the JSON data will be written.
    data (dict): The data to write in JSON format.
    append (bool): Whether to append the data to the existing file or overwrite it.
    """
    # Open the file in append mode ('a') if append is True, otherwise in write mode ('w')
    mode = 'a' if append else 'w'
    with open(output_path, mode) as file:
        # Write the data to the file in JSON format
        json.dump(data, file, indent=4)

def get_json_results(input_file_name: str, output_path:str, append = True):
    """ Runs inference on the dev set. Writes the results to a JSON file.
    Parameters:
    input_file_name (str): The name of the JSON file containing the data.
    output_path (str): The name of the file where the results will be written.
    append (bool): Whether to append the results to the existing file or overwrite it.
    Returns:
    None
    """
    claims = get_claims(input_file_name)
    for example in claims:
        example["questions"] = []  # Initialize the list if not already present
        for i in range(10):
            claim = example["claim"]

            # Tokenize the input text
            input_encodings = tokenizer(
                claim, max_length=512, truncation=True, padding="max_length", return_tensors="pt"
            )

            input_ids = input_encodings.input_ids
            attention_mask = input_encodings.attention_mask

            # Generate predictions with sampling
            outputs = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=512,
                num_return_sequences=1,
                do_sample=True,  # Enable sampling
                top_k=50,  # Use top-k sampling
                top_p=0.95,  # Use top-p (nucleus) sampling
                temperature=1.5,  # Adjust temperature for more diversity
                num_beams=5,  # Use beam search
                early_stopping=True
            )

            # Decode the output
            question_dict = {"question": tokenizer.decode(outputs[0], skip_special_tokens=True)}

            # Append the generated question to the list
            example["questions"].append(question_dict)

        # Print the example to verify the generated questions
        print(example)
    write_to_json_file(output_path, claims, append = True)
input_file_name = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/dev.json'
output_path = '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/10q, misinfo included/direct_question.json'
get_json_results(input_file_name, output_path, append = True)
print("Completed.")

{'claim': 'In a letter to Steve Jobs, Sean Connery refused to appear in an apple commercial.', 'questions': [{'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': "What was Sean Connery's response to the claim?"}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Did Sean Connery say "I don\'t want to appear in an apple commercial."'}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Did Sean Connery say that he refused to appear in an apple commercial?'}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}, {'question': 'Did Sean Connery say "I don\'t want to appear in an apple commercial."'}, {'question': 'Did Sean Connery refuse to appear in an apple commercial?'}]}
{'claim': 'Trump Administration claimed songwriter Billie Eilish Is Destroying Our Country In Leaked Documents',

In [ ]:
!pip install leven

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 4.0 MB/s eta 0:00:00
  Created wheel for leven: filename=leven-1.0.4-cp310-cp310-linux_x86_64.whl size=64147 sha256=ac04ed244670992de575fe93c22e10395ebad3a200213bbf507bbb8a82e364d7
  Stored in directory: /root/.cache/pip/wheels/a8/c7/c4/99e93cc37bb627f3d4d153d1a62feb3491c2e832a33e3ea2dc
Successfully built leven


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/AVeriTeC')
!python 'drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/Q_only_eval.py' --predictions '/content/drive/MyDrive/Colab Notebooks/Fact-checking explanations/Refined Code/10q, misinfo included/direct_question.json' --references '/content/drive/My Drive/Colab Notebooks/Fact-checking explanations/AVeriTeC/data/dev.json'

AVeriTeC evaluation:
Question-only score (HU-meteor):        0.45852360302727263
